In [ ]:
!pip install torchvision --upgrade

     |████████████████████████████████| 21.0 MB 201 kB/s 
     |████████████████████████████████| 750.6 MB 12 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1+cu111
    Uninstalling torchvision-0.11.1+cu111:
      Successfully uninstalled torchvision-0.11.1+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.11.0 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.11.0 which is incompatible.


In [ ]:
!mkdir StandfordDog

In [ ]:
import tarfile

with tarfile.open('/content/drive/MyDrive/FGVC-project/CUB_200_2011.tgz', 'r:gz') as tar:
  tar.extractall(path='/content')

with tarfile.open('/content/drive/MyDrive/FGVC-project/StandfordDog/images.tar', 'r') as tar:
  tar.extractall(path='/content/StandfordDog')

with tarfile.open('/content/drive/MyDrive/FGVC-project/StandfordDog/lists.tar', 'r') as tar:
  tar.extractall(path='/content/StandfordDog')


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torchvision.datasets import VisionDataset
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, Subset

import os 
from os import path

import numpy as np
import pandas as pd
from scipy.io import loadmat

from tqdm import tqdm
from PIL import Image


In [ ]:
class CUB2011(Dataset):
  def __init__(self, root, split_table, transform, train=True):

    fullset = torchvision.datasets.ImageFolder(root=root, transform=transform)

    if train:
      idxes = split_table.index[split_table['split'] == 1].tolist()
    else:
      idxes = split_table.index[split_table['split'] == 0].tolist()
    
    self.dataset = Subset(fullset, idxes)
    self.root = root
    self.transforms = fullset.transforms

  
  def __len__(self):
    return len(self.dataset)
  
  def __getitem__(self, idx):
    return self.dataset[idx][0], self.dataset[idx][1]
      

In [ ]:
split_table = pd.read_csv('/content/CUB_200_2011/train_test_split.txt', delimiter=' ', header=None, names=['id', 'split'])

In [ ]:
train_transforms = transforms.Compose([transforms.Resize((256, 256), Image.BILINEAR),
                                       transforms.RandomCrop((224, 224)),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
test_transforms = transforms.Compose([transforms.Resize((256, 256), Image.BILINEAR),
                                       transforms.CenterCrop((224, 224)),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:288: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


In [ ]:
train_dataset = CUB2011(root='/content/CUB_200_2011/images', split_table=split_table, transform=train_transforms, train=True)
test_dataset = CUB2011(root='/content/CUB_200_2011/images', split_table=split_table, transform=test_transforms, train=False)

In [ ]:

train_loader = DataLoader(train_dataset, batch_size=batch_size, 
                                              shuffle=True, num_workers=1)
test_loader = DataLoader(test_dataset, batch_size=batch_size, 
                                              shuffle=False, num_workers=1)

In [ ]:
next(iter(train_loader))

In [ ]:
class StandfordDog(Dataset):
  def __init__(self, root, split_idx, transform, train=True):

    fullset = torchvision.datasets.ImageFolder(root=root, transform=transform)

    train_idx, test_idx = split_idx

    if train:
      self.dataset = Subset(fullset, train_idx)
    else:
      self.dataset = Subset(fullset, test_idx)
    
    self.root = root
    self.transforms = fullset.transforms

  
  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    return self.dataset[idx][0], self.dataset[idx][1]

In [ ]:
file_list = loadmat('/content/StandfordDog/file_list.mat')
train_list = loadmat('/content/StandfordDog/train_list.mat')
test_list = loadmat('/content/StandfordDog/test_list.mat')

In [ ]:
files = [item[0][0] for item in file_list['file_list']]
train_data = [item[0][0] for item in train_list['file_list']]
test_data = [item[0][0] for item in test_list['file_list']]

In [ ]:
train_idx = []
test_idx = []
for i, file in enumerate(files):
  if file in train_data:
    train_idx.append(i)
  elif file in test_data:
    test_idx.append(i)

In [ ]:
split_idx = (train_idx, test_idx)

In [ ]:
train_dataset_sd = StandfordDog('/content/StandfordDog/Images', split_idx=split_idx, transform=train_transforms, train=True)
test_dataset_sd = StandfordDog('/content/StandfordDog/Images', split_idx=split_idx, transform=test_transforms, train=False)

In [ ]:
train_loader_sd = DataLoader(train_dataset_sd, batch_size=batch_size, 
                                              shuffle=True, num_workers=1)
test_loader_sd = DataLoader(test_dataset_sd, batch_size=batch_size, 
                                              shuffle=True, num_workers=1)

In [ ]:
next(iter(train_loader_sd))

In [ ]:
!pip install timm

     |████████████████████████████████| 431 kB 7.9 MB/s 


In [ ]:
!cp /content/drive/MyDrive/FGVC-project/Models/ResNet.py /content


In [ ]:
from ResNet import *
 

In [ ]:
model = ResNet(Bottleneck, 50, num_classes=1000, drop_path=0)

In [ ]:
state_dict = load_state_dict_from_url("https://download.pytorch.org/models/resnet50-0676ba61.pth", progress=True)

In [ ]:
model.load_state_dict(state_dict)

<All keys matched successfully>

In [ ]:
names = []
for name, params in model.named_parameters():
  names.append(name)

In [ ]:
from torchvision.models import resnet50

In [ ]:
torch_model = resnet50()
names_torch = []
for name, params in torch_model.named_parameters():
  names_torch.append(name)

In [ ]:
names[10:20], names_torch[10:20]

(['layer1.0.bn3.weight',
  'layer1.0.bn3.bias',
  'layer1.0.downsample.0.weight',
  'layer1.0.downsample.1.weight',
  'layer1.0.downsample.1.bias',
  'layer1.1.conv1.weight',
  'layer1.1.bn1.weight',
  'layer1.1.bn1.bias',
  'layer1.1.conv2.weight',
  'layer1.1.bn2.weight'],
 ['layer1.0.bn3.weight',
  'layer1.0.bn3.bias',
  'layer1.0.downsample.0.weight',
  'layer1.0.downsample.1.weight',
  'layer1.0.downsample.1.bias',
  'layer1.1.conv1.weight',
  'layer1.1.bn1.weight',
  'layer1.1.bn1.bias',
  'layer1.1.conv2.weight',
  'layer1.1.bn2.weight'])

In [ ]:
num_trainable_parameters = 0
for p in model.parameters():
    num_trainable_parameters += p.numel()
print("Number of Params: {}".format(num_trainable_parameters))

Number of Params: 25557032


In [ ]:
num_trainable_parameters = 0
for p in torch_model.parameters():
    num_trainable_parameters += p.numel()
print("Number of Params: {}".format(num_trainable_parameters))


Number of Params: 25557032


In [ ]:
model = ResNet(Bottleneck, 50, num_classes=200, drop_path=0)
model.cuda()

num_trainable_parameters = 0
for p in model.parameters():
    num_trainable_parameters += p.numel()
print("Number of Params: {}".format(num_trainable_parameters))

criterion = torch.nn.CrossEntropyLoss()
# optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=(len(train_loader) * epochs))
scaler = torch.cuda.amp.GradScaler()

Number of Params: 23944328


In [ ]:
def train(model, batch_size, train_loader, optimizer, criterion, scheduler, scaler):

    model.train()
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 

    num_correct = 0
    total_loss = 0

    for i, (x, y) in enumerate(train_loader):
        optimizer.zero_grad()

        x = x.cuda()
        y = y.cuda()

        with torch.cuda.amp.autocast():     
            outputs = model(x)
            loss = criterion(outputs, y)

        num_correct += int((torch.argmax(outputs, axis=1) == y).sum())
        total_loss += float(loss)

        batch_bar.set_postfix(
            acc="{:.04f}%".format(100 * num_correct / ((i + 1) * batch_size)),
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            num_correct=num_correct,
            lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))
         
        scaler.scale(loss).backward()
        scaler.step(optimizer) 
        scaler.update()

        scheduler.step() 
        batch_bar.update() 

    batch_bar.close() 

    train_accuracy = 100 * num_correct / (len(train_loader) * batch_size)
    train_loss = float(total_loss / len(train_loader))
    learning_rate = float(optimizer.param_groups[0]['lr'])

    return train_accuracy, train_loss, learning_rate

In [ ]:
def val(model, batch_size, val_loader, val_dataset):

    model.eval()
    batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')
    num_correct = 0
    for i, (x, y) in enumerate(val_loader):

        x = x.cuda()
        y = y.cuda()

        with torch.no_grad():
            outputs = model(x)

        num_correct += int((torch.argmax(outputs, axis=1) == y).sum())
        batch_bar.set_postfix(acc="{:.04f}%".format(100 * num_correct / ((i + 1) * batch_size)))

        batch_bar.update()
    
    batch_bar.close()

    val_accuracy = 100 * num_correct / len(val_dataset)

    return val_accuracy

In [ ]:
def model_ft(model):

    for name, param in model.named_parameters():
        if not name.startswith('fc'):
            param.requires_grad = False
    
    params_to_update = []
    for name, param in model.named_parameters():
        if param.requires_grad:
            params_to_update.append(param)
    
    return model, params_to_update

In [ ]:
for epoch in range(epochs):
  train_accuracy, train_loss, learning_rate = train(model, batch_size, train_loader, optimizer,criterion, scheduler, scaler)
  print("Epoch {}/{}: Train Acc {:.04f}%, Train Loss {:.04f}, Learning Rate {:.04f}".format(epoch + 1, epochs, train_accuracy, train_loss, learning_rate))

  if not (epoch + 1) % 10 and epoch > 0:
            val_accuracy = val(model, batch_size, test_loader, test_dataset)
            print("Validation: {:.04f}%".format(val_accuracy))
        

Train:   0%|          | 0/94 [00:01<?, ?it/s, acc=1.5625%, loss=5.5374, lr=0.0300, num_correct=1]/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Epoch 1/50: Train Acc 0.3324%, Train Loss 5.7715, Learning Rate 0.0300


Epoch 2/50: Train Acc 0.5652%, Train Loss 5.3165, Learning Rate 0.0299


Epoch 3/50: Train Acc 0.9309%, Train Loss 5.1905, Learning Rate 0.0297


Epoch 4/50: Train Acc 1.2301%, Train Loss 5.0951, Learning Rate 0.0295


...

Epoch 47/50: Train Acc 91.8717%, Train Loss 0.3501, Learning Rate 0.0003


Epoch 48/50: Train Acc 91.5891%, Train Loss 0.3414, Learning Rate 0.0001


Epoch 49/50: Train Acc 92.0379%, Train Loss 0.3318, Learning Rate 0.0000


Epoch 50/50: Train Acc 92.0711%, Train Loss 0.3300, Learning Rate 0.0000


Validation: 38.4536%


In [ ]:
model_pre = torchvision.models.resnet50(pretrained=True)
model_pre.fc = nn.Linear(2048, 200)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [ ]:
model_pre.to('cuda')
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model_pre.parameters(), lr=lr, momentum=0.9, weight_decay=1e-3)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=(len(train_loader) * epochs))
scaler = torch.cuda.amp.GradScaler()

In [ ]:
for epoch in range(epochs):
  train_accuracy, train_loss, learning_rate = train(model_pre, batch_size, train_loader, optimizer,criterion, scheduler, scaler)
  print("Epoch {}/{}: Train Acc {:.04f}%, Train Loss {:.04f}, Learning Rate {:.04f}".format(epoch + 1, epochs, train_accuracy, train_loss, learning_rate))

  if not (epoch + 1) % 10 and epoch > 0:
            val_accuracy = val(model_pre, batch_size, test_loader, test_dataset)
            print("Validation: {:.04f}%".format(val_accuracy))

Epoch 1/50: Train Acc 23.1383%, Train Loss 3.4015, Learning Rate 0.0300


Epoch 2/50: Train Acc 45.3624%, Train Loss 2.0021, Learning Rate 0.0299


Epoch 3/50: Train Acc 54.4382%, Train Loss 1.6088, Learning Rate 0.0297


Epoch 4/50: Train Acc 60.2394%, Train Loss 1.3801, Learning Rate 0.0295


Epoch 5/50: Train Acc 64.8770%, Train Loss 1.2060, Learning Rate 0.0293


...

Epoch 47/50: Train Acc 99.6343%, Train Loss 0.0340, Learning Rate 0.0003


Epoch 48/50: Train Acc 99.6343%, Train Loss 0.0342, Learning Rate 0.0001


Epoch 49/50: Train Acc 99.6343%, Train Loss 0.0343, Learning Rate 0.0000


Epoch 50/50: Train Acc 99.6343%, Train Loss 0.0340, Learning Rate 0.0000


Validation: 77.4594%


In [ ]:
val_accuracy = val(model_pre, batch_size, test_loader, test_dataset)

In [ ]:
convnext_tiny = torchvision.models.convnext_tiny(pretrained=True)
convnext_tiny.classifier[2] = nn.Linear(768, 200)

In [ ]:
epochs = 50
lr = 0.001
convnext_tiny.to('cuda')
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(convnext_tiny.parameters(), lr=lr, momentum=0.9, weight_decay=1e-3)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=(len(train_loader) * epochs))
scaler = torch.cuda.amp.GradScaler()

In [ ]:
epochs = 40
for epoch in range(epochs):
  train_accuracy, train_loss, learning_rate = train(convnext_tiny, batch_size, train_loader, optimizer,criterion, scheduler, scaler)
  print("Epoch {}/{}: Train Acc {:.04f}%, Train Loss {:.04f}, Learning Rate {:.04f}".format(epoch + 1, epochs, train_accuracy, train_loss, learning_rate))

  if not (epoch + 1) % 10 and epoch > 0:
            val_accuracy = val(convnext_tiny, batch_size, test_loader, test_dataset)
            print("Validation: {:.04f}%".format(val_accuracy))

Epoch 1/40: Train Acc 95.8278%, Train Loss 0.3124, Learning Rate 0.0000


Epoch 2/40: Train Acc 95.7613%, Train Loss 0.3078, Learning Rate 0.0000


Epoch 3/40: Train Acc 95.6948%, Train Loss 0.3211, Learning Rate 0.0000


Epoch 4/40: Train Acc 95.8444%, Train Loss 0.3191, Learning Rate 0.0000


Epoch 5/40: Train Acc 95.7945%, Train Loss 0.3133, Learning Rate 0.0000


...

Epoch 37/40: Train Acc 97.5898%, Train Loss 0.2067, Learning Rate 0.0008


Epoch 38/40: Train Acc 97.5898%, Train Loss 0.1941, Learning Rate 0.0009


Epoch 39/40: Train Acc 97.3737%, Train Loss 0.1988, Learning Rate 0.0009


Epoch 40/40: Train Acc 97.9555%, Train Loss 0.1804, Learning Rate 0.0009


Validation: 81.6362%


In [ ]:
val_accuracy = val(convnext_tiny, batch_size, test_loader, test_dataset)

In [ ]:
val_accuracy

73.48981705212289

In [ ]:
device = torch.device('cuda')

In [ ]:
device.type

'cuda'